<a href="https://colab.research.google.com/github/Jahid1990/Deep-Learning-Applications-R-Python/blob/master/Telecom_product_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load required packes and libraries

In [ ]:
#import cx_Oracle
import pandas as pd
import keras
import numpy as np
from keras import layers
from keras.layers import Input, LSTM, Dense,Conv1D,MaxPooling1D,Flatten,Embedding
from keras.layers import TimeDistributed
from keras.models import Model
from keras import metrics
from sklearn import preprocessing
import pickle

# Mount google drive in Colab. You can Ignore this part


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Saving & loading data to/from PC

In [ ]:
# with open(r'training_data_pack_reco_new.pickle', 'wb') as f:
#      pickle.dump([x_train,y_train,x_test,y_test], f)

In [ ]:
with open(r'/content/drive/My Drive/training_data_pack_reco_new_with_rcg_info.pickle', 'rb') as f:
    [x_train,y_train,x_test,y_test]= pickle.load(f)

# Model Building

In [ ]:
from keras.layers import Concatenate

inputs = Input(shape=(x_train.shape[1],))
shared=Dense(1024,activation='relu')(inputs)
shared=Dense(1024,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)
shared=Dense(512,activation='relu')(shared)

pack=Dense(512,activation='relu')(shared)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='relu')(pack)
pack=Dense(512,activation='tanh')(pack)
pack=Dense(16,activation='softmax',name='pack')(pack)

day=Dense(512,activation='relu')(shared)
day=Dense(512,activation='relu')(day)
day=Dense(512,activation='relu')(day)
day=Dense(512,activation='relu')(day)
day=Dense(256,activation='relu')(day)
day=Dense(128,activation='tanh')(day)
day=Dense(32,activation='softmax',name='day')(day)


model=Model(inputs=inputs,outputs=[pack,day])
#model.summary()

In [ ]:
from keras import optimizers
adm=optimizers.Adam( lr=0.001,beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
checkpoint = ModelCheckpoint('/content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min', 
                             period=1)

In [ ]:
#[x_test,[y_test[:,:16],y_test[:,16:]]]
model.compile(loss=['categorical_crossentropy','categorical_crossentropy'],optimizer='Adam', metrics=['accuracy'])
model.fit(x_train, [y_train[:,:16],y_train[:,16:]] ,epochs=100,validation_split=.2,verbose=1,batch_size=512,callbacks=[checkpoint])

In [ ]:
import numpy as np
import tensorflow as tf

np.set_printoptions(suppress=True)
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/pack_reco_engine_new_with_rcg_info.h5')

p=loaded_model.predict(x_test)

In [ ]:
pd.set_option('display.max_columns', 500)
pred=pd.DataFrame(p[0])
pred.head()

# Select TOP 3 

In [ ]:
top3=np.argsort(-p[0])[:,:3]
#pd.DataFrame(top3)
ytest=np.argmax(y_test[:,:16],axis=1)

val=[]
for i in range(y_test.shape[0]):
  if ytest[i] in top3[i]:
    val.append(1)
  else:
    val.append(0)

In [ ]:
pd.Series(val).value_counts()/(221887+27359)

1    0.890233
0    0.109767
dtype: float64

In [ ]:
pd.crosstab(val,ytest)

In [ ]:
pred_class=np.argmax(p[0], axis=1)

In [ ]:
actual=np.argmax(y_test[:,:16],axis=1)

In [ ]:
pd.crosstab(pred_class,actual)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(val,ytest))

In [ ]:
act=np.argmax(y_test[:,:16],axis=1)
pd.Series(act).value_counts()

In [ ]:
pd.Series(pred_class).value_counts()